In [673]:
from lusidtools.jupyter_tools import toggle_code
import os

"""Futures Valuation Workflow

Attributes
----------
futures
transaction types
recipes
valuations
"""

toggle_code("Toggle Docstring")

# Computing Intraday P&L for Futures with and without Daily Close outs

In this notebook, we demonstrate how P&L can be computed for Futures instruments under two different cash accounting treatments. We look at these treatments over the course of three days for both a Dec 21 and Mar 22 Bund Futures contract. Note that we maintain the same market quotes and transaction amounts for each Futures contract to more easily demonstrate how the PV and P&L measures evolve over time.


### Dec 21 Bund Futures with Unrealized P&L (‘non-close out’)
In our first example, we look at a Futures contract whereby the contract's daily P&L is left as unrealised on the position as opposed to marked-to-market on a daily basis.


### Mar 22 Bund Futures with Realized P&L ('close out’)
In our second example, we look at a Futures contract whereby the previous day's P&L is realized at the beginning of each day and booked down as variation margin. We then reset the cost basis of our holding to reflect this mark-to-market such that the computed PV at the start of each day is 0.

## Table of Contents
* [1. Create Portfolio](#-Create-Portfolio)
* [2. Create Futures](#-Create-Futures)
* [3. Transactions](#-Transactions)
* [4. Quotes](#-Quotes)
* [5. Valuations](#-Valuations)
* [6. A2B report](#-A2B-report)

In [674]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
from IPython.core.display import HTML
import uuid
# Import key modules from the LUSID package
import lusid
import lusid.models as models

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.utilities import create_scope_id
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidFuturesNotebook"
)

print("LUSID Environment Initialised")

print(

  "LUSID API Version :",

  api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised


In [ ]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
transaction_configuration_api = api_factory.build(lusid.api.TransactionConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

In [ ]:
# Define scopes
scope = "FuturesValuationNotebook"
quotes_scope = "FuturesValuationNotebook"
portfolio_code = "FutureWithNotionalCost"
recipeCode = "futuresValuation"
instruments_scope="FuturesValuationNotebook"
base_currency = "EUR"
transaction_type_scope = create_scope_id()

# 1. Create Portfolio<a class="anchor" id="-Create-Portfolio"></a>

In [ ]:
try:
    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency=base_currency,
            created="2010-01-01",
            sub_holding_keys=[],
            instrument_scopes=[instruments_scope]
        ),
    )

except lusid.ApiException as e:
    if(e.status == 401):
        print(e.reason)
    else:
        print(json.loads(e.body)["title"])


Could not create a portfolio with id 'FutureWithNotionalCost' because it already exists in scope 'ibor'.


# 2. Create Futures<a class="anchor" id="-Create-Futures"></a>

In [ ]:
# Define function that creates futures
def create_futures_contract(
        dom_ccy,
        contract_code,
        contract_month,
        contract_size,
        convention,
        country_id,
        fut_name,
        exchange_code,
        exchange_name,
        ticker_step,
        unit_value,
        ref_spot_price,
        start_date,
        maturity_date,
        fut_identifier,
):
    ctc = models.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        convention=convention,
        country=country_id,
        description=fut_name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
    )
    futuredef = models.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=ctc,
        contracts=1,
        ref_spot_price=ref_spot_price,
        underlying=models.ExoticInstrument(
            instrument_format=models.InstrumentDefinitionFormat(
                "custom", "custom", "0.0.0"
            ),
            content="{}",
            instrument_type="ExoticInstrument",
        ),
        instrument_type="Future",
    )
    # persist the instrument
    futureDefinition = models.InstrumentDefinition(
        name=fut_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(fut_identifier)},
        definition=futuredef,
    )
    batchUpsertRequest = {fut_identifier: futureDefinition}
    upsertResponse = instruments_api.upsert_instruments(request_body=batchUpsertRequest, scope=instruments_scope)
    futLuid = upsertResponse.values[fut_identifier].lusid_instrument_id
    print(futLuid)

## 2.1 Create Bund Futures Contract Expiring Dec 21

In [ ]:
start_date = datetime(2021, 3, 8, tzinfo=pytz.utc)
maturity_date = datetime(2021, 12, 8, tzinfo=pytz.utc)

dom_ccy = "EUR"
contract_code = "FGBL" #bbg=OE
contract_month = "Z"
contract_size = 100000
convention = "ActualActual"
country_id= "DE"
fut_name = "EURO-BUND FUTURE Dec21"
exchange_code = "EUREX"
exchange_name ="Eurex"
ticker_step = 0.01
unit_value = 10
ref_spot_price_val = None
identifier = "FutBund001"

# Create Futures Contract function
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

LUID_00006A64


## 2.2 Create Bund Futures Contract Expiring Mar 22

In [ ]:
start_date = datetime(2021, 6, 8, tzinfo=pytz.utc)
maturity_date = datetime(2022, 3, 8, tzinfo=pytz.utc)

dom_ccy = "EUR"
contract_code = "FGBL" #bbg=OE
contract_month = "H"
contract_size = 100000
convention = "ActualActual"
country_id= "DE"
fut_name = "EURO-BUND FUTURE Mar22"
exchange_code = "EUREX"
exchange_name ="Eurex"
ticker_step = 0.01
unit_value = 10
ref_spot_price_val = 0
identifier = "FutBund002"

# Create Futures Contract function
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

LUID_00007VI6


# 3. Transactions<a class="anchor" id="-Transactions"></a>

## 3.1 Create Transaction Types

To book Futures transactions under the close out and non-close out method, we need to create two sides and three transaction types. 

See documentation for sides [here](https://support.lusid.com/knowledgebase/article/KA-01875/en-us) and for transaction types [here](https://support.lusid.com/knowledgebase/article/KA-01749/en-us).

### 3.1.1 Sides
A side specifies what fields or properties of a transaction to use when generating holdings.

#### Notional
The definition for this side specifies the following:
  * **Security**: Use the `LusidInstrumentId` field from the transaction which will have the identifier of the instrument to book the trade against
  * **Currency**: Use the currency specified on the transaction as the currency of this trade
  * **Rate**: Use the rate specified on the transaction to convert between the trade currency and portfolio currency
  * **Units**: Use the `Units` fields on the transaction as the value for the number of securities traded
  * **Amount**: Use the `TotalConsideration` field on the transaction as the value for the value of this trade
  * **Notional amount**: Use the value on the property with the `Transaction/default/NotionalAmount` key on the transaction as the underlying value of the Future. Note that the property definition for this property is a system default. 

#### VM
Similar to the 'Notional' side, without using the notional amount; this side will be used to record the close out trade

### 3.1.2 Transaction Types

#### OpenContract

The first transaction type 'OpenContract' is a standard transaction used to enter into our Futures positions. This contains two movements. The first increases the size of the position based on a quantity provided, while the second decreases the cash position by the `TotalConsideration` amount on the transaction, used to record any fees for opening the contract.

#### CloseContract

The second transaction type 'CloseContract' is the same as 'OpenContract' in the opposite direction. 

#### VM

The third transaction type is 'VM' which is used to adjust the cost basis (variation margin) based on the position's daily P&L.
This contains two movements representing the two economic changes to the portfolio:
- The first movement updates the cost basis of the Futures position to reflect the daily mark-to-market change. This is done by adjusting the total_consideration of the position up or down while leaving the quantity of the position unchanged (in the transaction file we specify a quantity of 0). 
- The second movement records the cash generated from the daily mark-to-market change. This again gets its amount from the total_consideration value in the csv file.

In [ ]:
# Add default side definitions to the non-default transaction type scope.
# If working in the default scope, these side definitions are set by default so, unless these sides have been removed, this setting of sides can be skipped.
default_side_definitions = [
    models.SidesDefinitionRequest(
        side="Side1", 
        side_request=models.SideDefinitionRequest(
            security="Txn:LusidInstrumentId",
            currency="Txn:TradeCurrency",
            rate="Txn:TradeToPortfolioRate",
            units="Txn:Units",
            amount="Txn:TradeAmount")),
    models.SidesDefinitionRequest(
        side="Side2", 
        side_request=models.SideDefinitionRequest(
            security="Txn:SettleCcy",
            currency="Txn:SettlementCurrency",
            rate="SettledToPortfolioRate",
            units="Txn:TotalConsideration",
            amount="Txn:TotalConsideration"))
]

# Construct custom sides
side_list = [
    models.SidesDefinitionRequest(
        side="Notional", 
        side_request=models.SideDefinitionRequest(
            security="Txn:LusidInstrumentId",
            currency="Txn:TradeCurrency",
            rate="Txn:TradeToPortfolioRate",
            units="Txn:Units",
            amount="Txn:TotalConsideration",
            notional_amount="Transaction/default/NotionalAmount"
    )),
    models.SidesDefinitionRequest(
        side="VM",
        side_request=models.SideDefinitionRequest(
             security="Txn:LusidInstrumentId",
             currency="Txn:TradeCurrency",
             rate="Txn:TradeToPortfolioRate",
             units="Txn:Units",
             amount="Txn:TotalConsideration"
     ))
]

# Set all sides in the non-default transaction type scope
transaction_configuration_api.set_side_definitions(default_side_definitions + side_list, scope = transaction_type_scope)

transaction_type_req = [
   ("OpenContract", models.TransactionTypeRequest(
        aliases=[
            models.TransactionTypeAlias(
                type="OpenContract",
                description="Open a long future contract",
                transaction_class="Futures",
                transaction_roles="LongLonger",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types="StockMovement",
                side="Notional",
                direction=1,
                properties=None,
                mappings=[],
                name="Open contract"
            ),
            models.TransactionTypeMovement(
                movement_types="CashCommitment",
                side="Side2",
                direction=-1,
                properties=None,
                mappings=[],
                name="Fees"
            )
        ],
        properties=None,
    )),
    ("CloseContract", models.TransactionTypeRequest(
        aliases=[
            models.TransactionTypeAlias(
                type="CloseContract",
                description="Close a long future contract",
                transaction_class="Futures",
                transaction_roles="LongShorter",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types="StockMovement",
                side="Notional",
                direction=-1,
                properties=None,
                mappings=[],
                name="Close contract"
            ),
            models.TransactionTypeMovement(
                movement_types="CashCommitment",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[],
                name="Fees"
            )
        ],
        properties=None,
    )),
   ("VM", models.TransactionType(
        aliases=[
            models.TransactionTypeAlias(
                type="VM",
                description="Increase cost basis and adjust cash",
                transaction_class="Futures",
                transaction_roles="AllRoles",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types="VariationMargin",
                side="VM",
                direction=1,
                properties=None,
                mappings=[],
                name="Variation margin"
            ), models.TransactionTypeMovement(
                movement_types="CashReceivable",
                side="Side2",
                direction=1,
                properties=None,
                mappings=[],
                name="VM Received/Paid"
            )
        ],
        properties=None,
    ))
]

for(type, type_req) in transaction_type_req:
    transaction_configuration_api.set_transaction_type("default", type, type_req, scope=transaction_type_scope)
    print(f"{type} has been created in LUSID")

# Having created the transaction types in the custom scope, we update the transaction type scope of the portfolio
patch_document = [
    {
        "value": transaction_type_scope,
        "path": "/transactiontypescope",
        "op": "add"
    }
]
patch_response = api_factory.build(lusid.api.TransactionPortfoliosApi).patch_portfolio_details(
    scope=scope,
    code=portfolio_code,
    operation=patch_document)

Notional already exists in LUSID
VM already exists in LUSID
OpenContract has been created in LUSID
CloseContract has been created in LUSID
VM has been created in LUSID


## 3.2 Load Transaction Data

In [ ]:
# Read in transaction data read from file
futures_transactions = pd.read_csv("data/futures_notional_data.csv")

In [ ]:
transaction_requests=[
     models.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers={ "Instrument/default/ClientInternal": row["identifier"] },
                transaction_date=row["trade_date"],
                settlement_date=row["trade_date"],
                units=row["quantity"],
                transaction_price=models.TransactionPrice(
                    price=row["price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["total_consideration"], currency=base_currency
                ),
                properties={"Transaction/default/NotionalAmount": models.PerpetualProperty(
                    key="Transaction/default/NotionalAmount",
                value=models.PropertyValue(metric_value=models.MetricValue(value=row["notional_amount"],unit="EUR")))}
            ) for _ , row in futures_transactions.iterrows()
]

upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=scope,
        code=portfolio_code,
        transaction_request=transaction_requests)


# 4. Quotes<a class="anchor" id="-Quotes"></a>

## 4.1 Book Quotes

In [ ]:
# Read in futures data from file
futures_prices = pd.read_csv("data/futures_notional_prices.csv")
# Output futures data
futures_prices

,date,prices,identifier,id_type,currency,scaling_factor,note
0,2021-09-01T08:00:00Z,100.00,FutBund001,ClientInternal,EUR,100,Day 1 opening price
1,2021-09-01T12:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 1 intraday MTM price
2,2021-09-01T17:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 1 closing price
3,2021-09-02T08:00:00Z,100.50,FutBund001,ClientInternal,EUR,100,Day 2 opening price
4,2021-09-02T14:15:00Z,99.50,FutBund001,ClientInternal,EUR,100,Day2 intraday trade price
5,2021-09-02T17:00:00Z,98.00,FutBund001,ClientInternal,EUR,100,Day2 closing price
6,2021-09-03T08:00:00Z,98.00,FutBund001,ClientInternal,EUR,100,Day3 opening price
7,2021-09-03T14:15:00Z,104.00,FutBund001,ClientInternal,EUR,100,Day3 intraday trade price
8,2021-09-03T17:00:00Z,104.00,FutBund001,ClientInternal,EUR,100,Day3 closing price
9,2021-09-01T08:00:00Z,100.00,FutBund002,ClientInternal,EUR,100,Day1 opening price


In [ ]:
instrument_quotes = {
    index: models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["identifier"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid"),
            effective_at=row["date"]),
        metric_value=models.MetricValue(value=row["prices"], unit=row["currency"]),
        scale_factor=row["scaling_factor"]            
        ) for index, row in futures_prices.iterrows()
    }

#Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

# 5. Valuations<a class="anchor" id="-Valuations"></a>

## 5.1 Create valuation recipe

We use a simple recipe that uses the quotes upserted earlier in LUSID and a `ConstantTimeValueofMoney` pricing model that does not apply any discounting.

In [ ]:
# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code=recipeCode,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D",
            )
        ],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=models.PricingContext(
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueofMoney",
                instrument_type="Future",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 5.2 Create daily valuation function

The get_daily_fut_val() function when given a date & time along with a portfolio returns a valuation of the held instruments with an indicator of their price relative to the agreed future price. This valuation therefore enables you to know if your futures are worth more or less than you agreed to pay for them at any given time.

In [ ]:
def get_daily_fut_val(date, portfolio_code):
    
    metricsList=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            models.AggregateSpec("Instrument/Definition/ContractSize", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
            models.AggregateSpec("Holding/default/Units", "Value"),
            models.AggregateSpec("Holding/default/Cost", "Value"),
            models.AggregateSpec("Valuation/PV", "Value"),
            models.AggregateSpec("Valuation/Exposure/Amount", "Value"),
        
        ]
        
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=scope, code=recipeCode),
        metrics=metricsList,
      
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)
    
    columnsToRename={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Instrument/Definition/ContractSize": "Contract Size",
            "Quotes/Price": "Price",
            "Holding/default/Units": "Units",
            "Holding/default/Cost": "Cost",
            "Valuation/PV": "PV",
            "Valuation/Exposure/Amount": "Exposure",
    }      

    vals_df.rename(
        columns=columnsToRename,
        inplace=True,
    )

    return vals_df

## 5.3 Daily valuations

### Day 1

#### Start of Day
At 8:00am on Day 1, we enter into our two Bund Futures contracts with starting prices of 100.00. The notional amount is 1,000,000 for FutBund001, where  **notional_amount = price * quantity * contractSize**.

The *total_consideration* column accounts for the cost of open the contract which is EUR 30 for each contract.

In [ ]:
futures_transactions

,txn_id,txn_type,trade_date,quantity,identifier,price,total_consideration,notional_amount
0,fut_open_1,OpenContract,2021-09-01T08:00:00Z,10,FutBund001,100,30,1000000
1,fut_open_2,OpenContract,2021-09-01T08:00:00Z,5,FutBund002,100,30,500000
2,vm_1,VM,2021-09-02T08:00:00Z,0,FutBund001,0,5000,0
3,fut_close_1,CloseContract,2021-09-02T14:15:00Z,5,FutBund001,0,0,0
4,fut_close_2,CloseContract,2021-09-02T14:15:00Z,5,FutBund002,0,0,0
5,vm_2,VM,2021-09-03T08:00:00Z,0,FutBund001,0,-12500,0


We then immediately run a start of day valuation. Our positions have an exposure of EUR 1,000,000 for the Dec21 contract and EUR 500,000 for the Mar 22 one. 

PV is 0 for both contracts as there has been no price change since entering the contract.

The EUR -60.00 position reflects the cost to open the two contracts. 

In [ ]:
get_daily_fut_val("2021-09-01T08:00:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"1,000,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",100.00,10.00,30.00,0.00
1,-60.00,Euro,None,1.00,nan,-60.00,-60.00,-60.00
2,"500,000.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",100.00,5.00,30.00,0.00


#### Intraday

At 12:00pm noon, we decide to run an intraday mark to market valuation. We can see that the contract price for both instruments has moved from the original purchase price of 100.00, to 100.50. This gives us an intraday PV for the contracts of EUR 5,000 and EUR 2,500 respectively.

In [ ]:
get_daily_fut_val("2021-09-01T12:00:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"1,005,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",100.50,10.00,30.00,"5,000.00"
1,-60.00,Euro,None,1.00,nan,-60.00,-60.00,-60.00
2,"502,500.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",100.50,5.00,30.00,"2,500.00"


#### End of Day

At the end of the day, we see the contract prices remain at 100.50 yielding us the same valuation we produced intraday.

In [ ]:
get_daily_fut_val("2021-09-01T17:00:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"1,005,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",100.50,10.00,30.00,"5,000.00"
1,-60.00,Euro,None,1.00,nan,-60.00,-60.00,-60.00
2,"502,500.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",100.50,5.00,30.00,"2,500.00"


### Day 2
#### Start of Day

At 8:00am on Day 2, we book a 'VariationMargin' transaction in order to adjust the cost basis of the Dec 21 Bund Future upwards such that its start of day PV is 0. This adjustment can be automatically applied in LUSID removing the need to manually book these transactions each day.


In [ ]:
futures_transactions.iloc[2:3]

,txn_id,txn_type,trade_date,quantity,identifier,price,total_consideration,notional_amount
2,vm_1,VM,2021-09-02T08:00:00Z,0,FutBund001,0,5000,0


We again run a start of day valuation immediately afterwards and can see the Mar 22 Futures contract still has a PV of 2,500 EUR while the Dec 21 contract has a PV of 0. This is expected as we've now booked a cost basis adjustment upward in the amount of 5000 EUR for the Dec 21 contract. The Euro position has also increased by 5000 EUR.

In [ ]:
get_daily_fut_val("2021-09-02T08:00:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"1,005,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",100.50,10.00,30.00,0.00
1,"4,940.00",Euro,None,1.00,nan,"4,940.00","4,940.00","4,940.00"
2,"502,500.00",EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",100.50,5.00,30.00,"2,500.00"


#### Intraday

At 2:15pm, we decide to book a new transaction closing 5 contracts of each Future.

In [ ]:
futures_transactions.iloc[3:5]

,txn_id,txn_type,trade_date,quantity,identifier,price,total_consideration,notional_amount
3,fut_close_1,CloseContract,2021-09-02T14:15:00Z,5,FutBund001,0,0,0
4,fut_close_2,CloseContract,2021-09-02T14:15:00Z,5,FutBund002,0,0,0


We then run a valuation and see that the intraday PV of the Dec 22 contract is now -5000 EUR while the PV of the March 22 contract is now 0 as there are no contracts left. 

##### Dec 21 (PV) = (99.50 - 100.00) \* 10 \* 100,000 / 100 - (99.50 - 100.00) \* 10 \* 100,000 / 100 =  -5,000 EUR  
Where 100.00 is the transaction price of the first transaction and where 99.50 is the transaction price of the latest transaction.

<br/>

In [ ]:
get_daily_fut_val("2021-09-02T14:15:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"497,500.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",99.50,5.00,15.00,"-5,000.00"
1,"4,940.00",Euro,None,1.00,nan,"4,940.00","4,940.00","4,940.00"
2,0.00,EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",99.50,0.00,0.00,0.00


#### End of Day

At the close of the day, the future price has moved down further to 98.00. PVs is now computed as follows:

##### Dec 21 (PV) = (98.00 - 100.00) \* 5 \* 100,000 / 100 - 2,500 = -12,500 EUR
Where 100.00 is the cost basis of the first transaction and 2,500 is half of the variation margin transaction booked earlier.


In [ ]:
get_daily_fut_val("2021-09-02T17:00:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"490,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",98.00,5.00,15.00,"-12,500.00"
1,"4,940.00",Euro,None,1.00,nan,"4,940.00","4,940.00","4,940.00"
2,0.00,EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",98.00,0.00,0.00,0.00


### Day 3
#### Start of Day

At 8:00am on Day 3, we book a 'VariationMargin' transaction to adjust the cost basis of the Dec 22 Bund Future downwards in the amount of yesterday's P&L giving us a start of day PV of 0. 


In [ ]:
futures_transactions.iloc[5:6]

,txn_id,txn_type,trade_date,quantity,identifier,price,total_consideration,notional_amount
5,vm_2,VM,2021-09-03T08:00:00Z,0,FutBund001,0,-12500,0


We then take a start of day valuation of the two contracts and can see the Dec 21 contract has a PV of 0. We've updated today's cash by subtracting yesterday's PV of -12,500 EUR from yesterdays cash balance of 4,940 EUR to arrive at today's new cash balance of -7,560 EUR.
<br/><br/>
At the start of the day, we take a valuation with a price of 98.00

In [ ]:
get_daily_fut_val("2021-09-03T08:00:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"490,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",98.00,5.00,15.00,0.00
1,"-7,560.00",Euro,None,1.00,nan,"-7,560.00","-7,560.00","-7,560.00"
2,0.00,EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",98.00,0.00,0.00,0.00


#### Intraday

Intraday, we take a valuation of our portfolio where the price has moved to 104.00

##### Dec 21 (PV) = (104.00 - 98) * 5 * 100,00/100 = 30,000 EUR
Where 98 is yesterday's price

In [ ]:
get_daily_fut_val("2021-09-03T14:15:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"520,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",104.00,5.00,15.00,"30,000.00"
1,"-7,560.00",Euro,None,1.00,nan,"-7,560.00","-7,560.00","-7,560.00"
2,0.00,EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",104.00,0.00,0.00,0.00


#### End of Day

Similar to our intraday valuation, we see the price hasn't changed at 104.00 and take a final end of day valuation

##### Dec 21 (PV) = (104.00 - 98) * 5 * 100,00/100 = 30,000 EUR
Where 98 is yesterday's price

In [ ]:
get_daily_fut_val("2021-09-03T17:00:00Z", portfolio_code)

,Exposure,InstrumentName,ClientInternal,Contract Size,Price,Units,Cost,PV
0,"520,000.00",EURO-BUND FUTURE Dec21,FutBund001,"100,000.00",104.00,5.00,15.00,"30,000.00"
1,"-7,560.00",Euro,None,1.00,nan,"-7,560.00","-7,560.00","-7,560.00"
2,0.00,EURO-BUND FUTURE Mar22,FutBund002,"100,000.00",104.00,0.00,0.00,0.00


# 6. A2B Report <a class="anchor" id="-A2B-report"></a>

In [ ]:
# Helper function for displaying A2B Report
def format_df(response):
    a2b_df = lusid_response_to_data_frame(response, rename_properties=True)
    
    # Reduce column properties
    exclude = ["instrument_uid", "sub_holding_keys","currency","group_id", "portfolio_id.scope","portfolio_id.code", "holding_type", "instrument_scope", "end.portfolio_currency.components"]
    
    # Drop results in holding currency and results that indicate currency of the activity; we know that everything is in EUR in this example
    cols_to_drop = [col for col in a2b_df.columns for exclusion in exclude if (exclusion == col) or ("holding_currency" in col) or ("currency.currency" in col)]
    
    result = a2b_df.drop(columns=cols_to_drop)
    
    columns_to_rename = {
        "Name(default-Properties)": "Name",
       
        "start.portfolio_currency.components.Cost": "Start/Cost",
        "start.portfolio_currency.components.MarketGain": "Start/Market Gain",
        "start.portfolio_currency.total": "Start/Total",
        "gains.portfolio_currency.components.DeltaMarketGain": "Gains/Delta Market Gain",
        "gains.portfolio_currency.total": "Gains/Total",
        "flows.portfolio_currency.components.VM Received/Paid": "Flow/VM Received/Paid",
        "flows.portfolio_currency.components.Variation margin": "Flow/Variation Margin",
        "flows.portfolio_currency.total": "Flows/Total",
        "end.portfolio_currency.components.Cost": "End/Cost",
        "end.portfolio_currency.components.MarketGain": "End/Market Gain",
        "end.portfolio_currency.components.VariationMargin": "End/Variation Margin",
        "end.portfolio_currency.total": "End/Total",

    }
    result=result[columns_to_rename.keys()]
    result.rename(columns=columns_to_rename, inplace=True)
    return result

For breaking down the profit and loss occurred in a given period, LUSID offers an A2B report, see [here](https://www.lusid.com/docs/api#operation/GetA2BData) for documentation.

The AtoB report will give a breakdown of each holding's P&L, including the flows (transactions) that have occurred in the selected time window as well as totals at the start and end of the window. Within the report you will find the following details:

 - **Market Value(Start)**: The market value at the start of the window.
 - **Gains**: Capital Gain, or gain due to asset appreciation based on available market data.
 - **Flows**: Any transaction activity occurring in the period.
 
Note, the following table has been filtered down to some core fields.

### 6.1 A2B breakdown
The A2B time window runs between 1st of September 2021 at 08:00 and 2nd of September 2021 at 14:00. That is, from the time of contract open to right before the contract close. The following components are returned:

* **Start**:
  * EUR : The start position for cash is a cost of EUR -60.00 which is the sum of the contract open fees. There is no market gain as there were no previous positions and the contract were opened in the same currency as the portfolio, so no FX P&L.
  * The 2 contracts have the same starting values: a cost of EUR 30.00 each and equivalent offset in market gain.
* **Gains and flows**:
  * EUR: No gains to be seen as there's no FX P&L
  * Dec 21 contract: There is a EUR 5,000 Market Gain and an offset in cash of EUR -5,000; the flow columns show that it's the Variation Margin trade
  * Mar 22 contract: There is a EUR 2,500 Market gain with no offsetting flows because there was no Variation Margin trade booked against this contract. The gain reflects the price increase from the start of the contract up to the end of the time window 
* **End**
  * The end columns show a sum of all the activities for each position

In [ ]:
# Request A2B report
a2b = transaction_portfolio_api.get_a2_b_data(
    scope=scope,
    code=portfolio_code,
    from_effective_at=datetime(2021, 9, 1, 8, tzinfo=pytz.utc),
    to_effective_at=datetime(2021, 9, 2, 14, tzinfo=pytz.utc),
    recipe_id_scope=scope,
    recipe_id_code=recipeCode,
    property_keys=["Instrument/default/Name"]
)

a2b_df = format_df(a2b)
a2b_df

,Name,Start/Cost,Start/Market Gain,Start/Total,Gains/Delta Market Gain,Gains/Total,Flow/VM Received/Paid,Flow/Variation Margin,Flows/Total,End/Cost,End/Market Gain,End/Variation Margin,End/Total
0,Euro,-60.00,nan,-60.00,nan,nan,"5,000.00",nan,"5,000.00","4,940.00",nan,nan,"4,940.00"
1,EURO-BUND FUTURE Dec21,30.00,-30.00,0.00,"5,000.00","5,000.00",nan,"-5,000.00","-5,000.00",30.00,"4,970.00","-5,000.00",0.00
2,EURO-BUND FUTURE Mar22,30.00,-30.00,0.00,"2,500.00","2,500.00",nan,nan,nan,30.00,"2,470.00",nan,"2,500.00"
